<a href="https://colab.research.google.com/github/ChavChavC/BT4222/blob/main/Feature_Engineering1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install torch
!pip install transformers
!pip install evaluate

In [3]:
import torch
torch.cuda.current_device()

0

In [4]:
from datasets import load_dataset
from torch.utils.data import DataLoader
dataset = load_dataset("Jean-Baptiste/financial_news_sentiment")

test

Roberta

In [5]:
# # ROBERTA FIN BERT model
# #Their own model
# from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
# model = AutoModelForSequenceClassification.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")
# tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")

# classifier=pipeline("text-classification",model=model, tokenizer=tokenizer)
# output=classifier(dataset['train']['summary_detail_with_title'][0])
# print(output)

FinBERT

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# output=classifier(dataset['train']['summary_detail_with_title'][0])
# print(output)

BERT/ Distilbert

In [6]:
modelname = "bert-base-uncased"
modelname = "distilbert-base-uncased"
modelname = "distilbert-base-uncased"


# modelname = "distilbert-base-uncased"
# id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2:"POSITIVE"}
# label2id = {"NEGATIVE": 0, "NEUTRAL":1 , "POSITIVE": 2}
# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
# )

In [7]:
# Load the BERT tokenizer from the "bert-base-uncased" pre-trained model
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(modelname)
def tokenize_function(examples):
# Tokenize the "text" column of the examples, adding padding to the maximum length and truncating if necessary
        return tokenizer(examples["summary_detail_with_title"], padding="max_length", truncation=True)
# Apply the tokenize function to the dataset in batches
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["summary_detail_with_title"])
# Rename the "label" column to "labels" to match the expected format for training
tokenized_datasets.set_format("torch")
# Shuffle the dataset with a fixed seed and select a range of examples
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

In [8]:
train_dataset = train_dataset.remove_columns(['summary_detail', 'title', 'topic',  '__index_level_0__',])
eval_dataset = eval_dataset.remove_columns(['summary_detail', 'title', 'topic',  '__index_level_0__',])

In [9]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
batchsize=4
# Create Dataloader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batchsize)
eval_dataloader = DataLoader(eval_dataset, batch_size=batchsize)

In [12]:
from transformers import AutoModelForMaskedLM
# Because we initialized BertForMaskedLM and concat is with our classifier instead of directly using BertForSequenceClassification
# Some weights of the model checkpoint at bert-base-uncased were not used is within the expectation.
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW


class Model(nn.Module):
    def __init__(self,output_dim,dropout_rate):
        super(Model,self).__init__()
        self.encoder=AutoModelForMaskedLM.from_pretrained(modelname, output_hidden_states=True, return_dict=True)
        self.dropout=nn.Dropout(dropout_rate)
        # For the "bert-base-uncased" model, each hidden state has a dimension of 768.
        # the value 3072=4*768 corresponds to the total dimension of the concatenated hidden states from the BERT model.
        self.classifier=nn.Linear(3072,output_dim)


    def forward(self,input_ids,token_type_ids,attention_mask):
        outputs = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        hidden_states = torch.cat(tuple([outputs.hidden_states[i] for i in [-1, -2, -3, -4]]), dim=-1) # [bs, seq_len, hidden_dim*4]
        # We are actually extracting the hidden state of the [CLS] token for each sequence in the batch.
        # This [CLS] token's hidden state is typically used as a fixed-size representation of the entire sequence.
        # This representation has been learned during BERT's pretraining to capture important information for various tasks.
        # In the context of classification, you can think of the [CLS] token's hidden state as a summary of the sequence's content,
        # which is then fed into the linear classifier to make predictions for the task at hand.
        x=self.dropout(hidden_states[:, 0, :])
        x=self.classifier(x)
        return x

model = Model(output_dim=3, dropout_rate = 0.5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fct = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import get_scheduler
from tqdm.auto import tqdm
import evaluate

epochs = 2
num_training_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )
metric = evaluate.load("accuracy")
progress_bar = tqdm(range(num_training_steps))


for epoch in range(epochs):
        for batch in train_dataloader:
            model.train()
            # Loop through batches in the training data loader
            label_ids = batch['labels']
            input_ids = batch['input_ids']
            token_type_ids = None
            # When using BERT for tasks like single-text classification or sequence labeling, the token_type_ids is an optional parameter, commonly set to None.
            attention_mask = batch['attention_mask']
            # Perform a forward pass through the model to get logits
            logits = model(input_ids, token_type_ids, attention_mask)

            # Calculate the loss using the provided loss function
            loss = loss_fct(logits, label_ids.view(-1))
            # Perform backward pass and update model parameters
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad() # Clear accumulated gradients
            progress_bar.update(1) # Update progress bar

        # Set the model to evaluation mode for validation
        model.eval()
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad(): # disable gradient computation
                label_ids = batch['labels']
                input_ids = batch['input_ids']
                token_type_ids = None
                attention_mask = batch['attention_mask']
                logits = model(input_ids, token_type_ids, attention_mask)
                loss = loss_fct(logits, label_ids.view(-1))

            # Get predicted labels by selecting the class with the highest probability
            predictions = torch.argmax(logits, dim=-1)
            metric.add_batch(predictions=predictions, references=batch["labels"])

        acc = metric.compute()
        print(f'Epoch {epoch+1}')
        print(f'val_loss : {loss}')
        print(f"val_accuracy: {acc['accuracy'] * 100}")
        print(25*'==')

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# # D
# #Their own model
# from transformers import DistilBertTokenizer, DistilBertModel
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer,DataCollatorWithPadding
# import numpy as np
# import evaluate

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2:"POSITIVE"}
# label2id = {"NEGATIVE": 0, "NEUTRAL":1 , "POSITIVE": 2}


# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
# )

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# accuracy = evaluate.load("accuracy")
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

# training_args = TrainingArguments(
#     output_dir="my_awesome_model",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     tokenizer=tokenizer,
#     data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
#     compute_metrics=compute_metrics,
# )

# trainer.train()

In [ ]:
# classifier=pipeline("text-classification",model=model, tokenizer=tokenizer)
# output=classifier(dataset['train']['summary_detail_with_title'][0])
# print(output)